In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from joblib import Parallel, delayed

import sqlite3
import sys
import time
import math
#import tqdm
from tqdm.auto import tqdm
import datetime
import os
import pickle
from pathlib import Path

from glicko2 import Player
import multiprocessing

tqdm.pandas()

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [ ]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

if os.path.exists(data_path + 'dfs_dict.pkl'):
    cell_has_run = True
    # Load the dictionary of DataFrames from the pickle
    with open(data_path + 'dfs_dict.pkl', 'rb') as f:
        dfs = pickle.load(f)
# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = data_path + "melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [ ]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [ ]:
players_df = dfs['players_df']
players_df.head()

In [ ]:
ranking_df = dfs['ranking_df']
ranking_df.head()

In [ ]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

In [ ]:
sets_df = dfs['sets_df']
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data)")

sets_df.head()

In [ ]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

In [ ]:
# Code optimization by Dan
# Basically we want to replace this line in process_tournament with something more efficient:
#
#      tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
#
# Instead, we can
# - Merge the tournament date info into ``sets_df``
# - Sort by date
# - Store the start/end positions of each tournament in a separate dictionary
# - Use tournament_sets_df = sets_df.iloc[start:end+1] instead.

sets_df = sets_df.merge(tournament_info_df[['key', 'start', 'end']], left_on='tournament_key', right_on='key', how='left')
sets_df = sets_df.drop(labels=['key_y'], axis='columns')
sets_df = sets_df.rename(columns={"key_x": "key"})
sets_df = sets_df.sort_values(by=['end', 'tournament_key']) # Just in case there are tournaments with the exact same end date

In [ ]:
# A bit of data cleanup
# TODO: Rerun!
min_date = datetime.datetime(2015, 1, 1)
max_date = datetime.datetime(2024, 12, 31)

sets_df = sets_df[(sets_df['start'] >= min_date) & (sets_df['end'] >= min_date) & (sets_df['start'] <= max_date) & (sets_df['end'] <= max_date)]
sets_df[sets_df['end'] <= datetime.datetime(2017,1,1)]

In [ ]:
# For now we ignore the L{n} location name.
top_8_locations = [                                   
        ['WSF', 'Winners Semis', 'Winners Semi-Final'],
        ['LQF', 'Losers Quarters', 'Losers Quarter-Final'],
        ['WF', 'Winners Final', 'Winners Final'],
        ['LSF', 'Losers Semis', 'Losers Semi-Final'],
        ['LF', 'Losers Final', 'Losers Final'],
        ['GF', 'Grand Final', 'Grand Final'],
        ['GFR', 'GF Reset', 'Grand Final Reset']
    ] 

top_8 = sets_df['location_names'].isin(top_8_locations)

In [ ]:
dataset_mini_df = pd.read_pickle(data_path + 'dataset_mini.pkl')

# Temporary bugfix, might have added stuff twice at some point
# dataset_mini_df = dataset_mini_df.loc[:,~dataset_mini_df.columns.duplicated()].copy()

#minier_cols = [x for x in dataset_mini_df.columns if "m2" not in x and "_alt_" not in x]
minier_cols = [x for x in dataset_mini_df.columns if "m2" not in x and "_alt_" not in x]
#minier_cols = [x for x in dataset_mini_df.columns if "m2" not in x]
dataset_minier_df = dataset_mini_df[minier_cols]

dataset_df = dataset_minier_df

# Optionally restrict to top 8, higher quality data presumably
# Note that we are assuming that dataset_df and sets_df share the same actual sets, row by row, just with different computed info!
dataset_df = dataset_df[top_8]

# Throw away old data, probably
dataset_df = dataset_df.iloc[40000:]

# Sets with real data
dataset_df = dataset_df[(dataset_df['p1_elo'] != 1500.0) &
                        (dataset_df['p2_elo'] != 1500.0) &
                        (dataset_df['p1/m1/m1_elo'] != 1500.0) &
                        (dataset_df['p2/m1/m1_elo'] != 1500.0) &
                        (dataset_df['p1/m1_elo'] != 1500.0) &
                        (dataset_df['p2/m1_elo'] != 1500.0)]
dataset_df

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset_df[dataset_df.columns[:-1]]
y = dataset_df[dataset_df.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt

X_train_win = X_train[y_train == 1.0]
X_train_loss = X_train[y_train == 0.0]

# 'p1_m1_usage', 'p2_m1_usage', 'p1/m1/m1_elo', 'p1/m1/m1_updates', 'p2/m1/m1_elo', 'p2/m1/m1_updates'


plt.scatter(X_train_win['p1_elo'], X_train_win['p2_elo'], s=0.3)
plt.scatter(X_train_loss['p1_elo'], X_train_loss['p2_elo'], s=0.3)
plt.xlabel('p1_elo')
plt.ylabel('p2_elo')

In [ ]:
features_rd = ['p1_rd', 'p2_rd', 'p1/m1/m1_rd', 'p2/m1/m1_rd', 'p1/m1_rd', 'p2/m1_rd']
X_train[features_rd].iloc[0].values ** 2

In [ ]:
import importlib
import errorlda
from sklearn.metrics import accuracy_score, log_loss
import random

importlib.reload(errorlda)

features_elo = ['p1_elo', 'p2_elo', 'p1/m1/m1_elo', 'p2/m1/m1_elo', 'p1/m1_elo', 'p2/m1_elo']
features_rd = ['p1_rd', 'p2_rd', 'p1/m1/m1_rd', 'p2/m1/m1_rd', 'p1/m1_rd', 'p2/m1_rd']
scaling = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

X_elo_train = X_train[features_elo]
X_elo_train_errors = X_train[features_rd].apply(lambda row: np.diag(np.multiply(row.values ** 2, scaling)), axis=1)

X_elo_test = X_test[features_elo]
X_elo_test_errors = X_test[features_rd].apply(lambda row: np.diag(np.multiply(row.values ** 2, scaling)), axis=1)

train_indices = random.sample(list(X_elo_train.index), 10000)

model = errorlda.ErrorLDA()
model.fit(X_elo_train.loc[train_indices], y_train.loc[train_indices],
          X_train_errors=X_elo_train_errors.loc[train_indices], error_scaling=True)

#model.predict_proba_one([1900.0, 1500.0], x_error=np.diag([300.0, 300.0])**2)
y_prob = model.predict_proba(X_elo_test, X_error=X_elo_test_errors)
y_pred = (y_prob[1.0] >= 0.5).astype(float)

print(log_loss(y_test, y_prob))
print(accuracy_score(y_test, y_pred))
print()

In [ ]:
# To be used for testing a faulty model
y_prob = model.predict_proba(X_elo_test, X_error=X_elo_test_errors)
y_pred = (y_prob[1.0] >= 0.5).astype(float)

print(log_loss(y_test, y_prob))
print(accuracy_score(y_test, y_pred))
print()

In [ ]:
model.variance += np.identity(len(features_rd))
model.debug_func()

In [ ]:
# MIN LOG LOSS / MAX ACCURACY
# global elo: ??? / 0.45
# alt2:       0.65 / 0.9
# alt3:       0.5 / 0.9
#
# global and alt2 have slightly decreasing accuracy just before reaching min log loss

import importlib
import errorlda
from sklearn.metrics import accuracy_score, log_loss
import random

importlib.reload(errorlda)

#features_elo = ['p1_elo', 'p2_elo', 'p1/m1/m1_elo', 'p2/m1/m1_elo', 'p1/m1_elo', 'p2/m1_elo']
#features_rd = ['p1_rd', 'p2_rd', 'p1/m1/m1_rd', 'p2/m1/m1_rd', 'p1/m1_rd', 'p2/m1_rd']

features_elo = ['p1_elo', 'p2_elo']
features_rd = ['p1_rd', 'p2_rd']

num_test_vals = 10
samples_per_train = 20000
outcomes = np.zeros(shape=(num_test_vals, 2)) # log loss, accuracy
test_vals = np.linspace(0.1, 1.0, num_test_vals)

for i, scale_variance in enumerate(tqdm(test_vals)):

    X_elo_train = X_train[features_elo]
    X_elo_train_errors = X_train[features_rd].apply(lambda row: np.diag(row.values ** 2) * scale_variance, axis=1)

    X_elo_test = X_test[features_elo]
    X_elo_test_errors = X_test[features_rd].apply(lambda row: np.diag(row.values ** 2) * scale_variance, axis=1)

    train_indices = random.sample(list(X_elo_train.index), samples_per_train)

    model = errorlda.ErrorLDA()
    model.fit(X_elo_train.loc[train_indices], y_train.loc[train_indices], X_train_errors=X_elo_train_errors.loc[train_indices])

    y_prob = model.predict_proba(X_elo_test, X_error=X_elo_test_errors)
    y_pred = (y_prob[1.0] >= 0.5).astype(float)

    outcomes[i, 0] = log_loss(y_test, y_prob)
    outcomes[i, 1] = accuracy_score(y_test, y_pred)

# Actually see exactly where optimal values occur
acc_max_x = np.argmax(outcomes[:,1])
loss_min_x = np.argmin(outcomes[:,0])

plt.subplot(1,2,1)
plt.plot(test_vals, outcomes[:,1], label='Accuracy')
plt.scatter(test_vals[acc_max_x], outcomes[acc_max_x,1])
plt.xlabel('Scaling factor')
plt.legend()

plt.subplot(1,2,2)
plt.plot(test_vals, outcomes[:,0], label='Log loss')
plt.scatter(test_vals[loss_min_x], outcomes[loss_min_x,0])
plt.xlabel('Scaling factor')
plt.legend()

In [ ]:
import errorlda
importlib.reload(errorlda)

model = errorlda.ErrorLDA()

print(len(X_train.index))

model.fit(X_train[['p1_elo', 'p2_elo']].iloc[10000:20000], y_train.iloc[10000:20000],
          X_train_errors=X_train[['p1_rd', 'p2_rd']].iloc[10000:20000].apply(lambda row: np.diag(row.values ** 2), axis=1))

In [ ]:
matrices = np.array([[[1,2],[3,4]],[[5,6],[7,8]],[[5,6],[7,8]]])
vectors = np.array([[10,15],[20,25],[30,35]])

np.einsum('ij,ijk,ik->i', vectors, matrices, vectors).sum()

In [ ]:
# JUST the global elos
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss

model = LogisticRegression(penalty=None)
model.fit(X_train[['p1_elo', 'p2_elo']], y_train)

print(log_loss(y_test, model.predict_proba(X_test[['p1_elo', 'p2_elo']])))
print(accuracy_score(y_test, model.predict(X_test[['p1_elo', 'p2_elo']])))

In [ ]:
# JUST the global elos
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
import random

all_elos = ['p1_elo', 'p2_elo', 'p1/m1_elo', 'p2/m1_elo', 'p1/m1/m1_elo', 'p2/m1/m1_elo']
train_indices = random.sample(list(X_train.index), 10000)

model = LogisticRegression(penalty=None, max_iter=10000)
#model.fit(X_train.loc[train_indices][all_elos], y_train.loc[train_indices])
model.fit(X_train[all_elos], y_train)

print(log_loss(y_test, model.predict_proba(X_test[all_elos])))
print(accuracy_score(y_test, model.predict(X_test[all_elos])))

In [ ]:
# JUST the global elos
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, log_loss

model = LinearDiscriminantAnalysis()
model.fit(X_train[['p1_elo', 'p2_elo', 'p1/m1_elo', 'p2/m1_elo', 'p1/m1/m1_elo', 'p2/m1/m1_elo']], y_train)

print(log_loss(y_test, model.predict_proba(X_test[['p1_elo', 'p2_elo', 'p1/m1_elo', 'p2/m1_elo', 'p1/m1/m1_elo', 'p2/m1/m1_elo']])))
print(accuracy_score(y_test, model.predict(X_test[['p1_elo', 'p2_elo', 'p1/m1_elo', 'p2/m1_elo', 'p1/m1/m1_elo', 'p2/m1/m1_elo']])))